<a href="https://colab.research.google.com/gist/MBcode/196bfc15f775531ecd1c161b58efb3f7/ingesttesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GeoCODES ingest pipeline testing:

repo_counts(from sitemap) -> repo_ld_counts(from LD-cache) -> final_counts(in the endpoint)

Then unit/end-to-end spot testing of crawl products, to check that code changes don't change outputs

#Load the github.com/earthcube/earthcube_utilities 
 for now the staging version

In [2]:
%load_ext rpy2.ipython
%load_ext google.colab.data_table
import httpimport
with httpimport.github_repo('MBcode', 'ec'):
  import ec

# Start by seeing how many records are expected from each repository
To be harvested by google-datasetsearch or GeoCODES only happens if the repository has a 'sitemap.xml' listing the dataset landing pages, which have jsonld in them, that they want indexed, so the search engine can find dataset resources of interest So we start by crawling the sitemaps, (but just this time) we will only record the number of records, so we can know how many to expect from each repository, during in the next parts of the ingestion pipeline

In [7]:
sitemaps=[
"https://www.unavco.org/data/doi/sitemap.xml",
"https://ssdb.iodp.org/dataset/sitemap.xml",
#"http://balto.opendap.org/opendap/site_map.txt ",
"http://wiki.linked.earth/sitemap.xml",
"https://lipdverse.org/sitemap.xml",
"http://ds.iris.edu/files/sitemap.xml",
"https://data.ucar.edu/sitemap.xml",
"http://opencoredata.org/sitemap.xml",
"https://www2.earthref.org/MagIC/contributions.sitemap.xml",
#"http://data.neotomadb.org/sitemap.xml",
"https://ecl.earthchem.org/sitemap.xml",
"https://xdomes.tamucc.edu/srr/sensorML/sitemap.xml",
"https://geodex.org/neon_prodcodes_sm.xml",
#"https://www.designsafe-ci.org/sitemap.xml ",
#"https://object.cloud.sdsc.edu/v1/AUTH_85f46aa78936477d8e71b186269414e8/gleaner-summoned",
#"http://datadiscoverystudio.org/sitemap/CinergiSiteIndex.xml",
"https://www.unidata.ucar.edu/sitemap.xml",
"https://service-dev.rvdata.us/api/sitemap/",
"https://raw.githubusercontent.com/earthcube/GeoCODES-Metadata/gh-pages/metadata/Dataset/sitemap.xml",
"https://www.usap-dc.org/view/dataset/sitemap.xml",
"https://cchdo.ucsd.edu/sitemap.xml",
"https://amgeo-dev.colorado.edu/sitemap.xml"]

In [4]:
ec.setup_sitemap()
repos=["amgeo","cchodo","designsafe","earthchem","geocodes_demo_dataset","iris","linked.earth",
"lipdverse","magic", "opencoredata","r2r","ssdb.iodp","ucar","unavco","unidata","usap-dc"]

In [4]:
#sitemaps_count = ec.sitemaps_count(sitemaps)
#sitemaps_count  #will used cached values for now

In [ ]:
sitemap_count

{'http://datadiscoverystudio.org/sitemap/CinergiSiteIndex.xml': 0,
 'http://ds.iris.edu/files/sitemap.xml': 0,
 'http://opencoredata.org/sitemap.xml': 83388,
 'http://wiki.linked.earth/sitemap.xml': 18634,
 'https://cchdo.ucsd.edu/sitemap.xml': 5040,
 'https://data.ucar.edu/sitemap.xml': 17005,
 'https://ecl.earthchem.org/sitemap.xml': 920,
 'https://geodex.org/neon_prodcodes_sm.xml': 0,
 'https://lipdverse.org/sitemap.xml': 704,
 'https://object.cloud.sdsc.edu/v1/AUTH_85f46aa78936477d8e71b186269414e8/gleaner-summoned': 0,
 'https://raw.githubusercontent.com/earthcube/GeoCODES-Metadata/gh-pages/metadata/Dataset/sitemap.xml': 0,
 'https://service-dev.rvdata.us/api/sitemap/': 0,
 'https://ssdb.iodp.org/dataset/sitemap.xml': 25241,
 'https://www.unavco.org/data/doi/sitemap.xml': 0,
 'https://www.unidata.ucar.edu/sitemap.xml': 211,
 'https://www.usap-dc.org/view/dataset/sitemap.xml': 0,
 'https://www2.earthref.org/MagIC/contributions.sitemap.xml': 0,
 'https://xdomes.tamucc.edu/srr/sensorM

# Next see how many made it into our LinkedData-cache
Once we capture the jsonld from each landing page, we cache that LinkedData as is, and then in another RDF format: .nt ntriples, that is easier to concatenate and to load into a triplestore

In [ ]:
ec.setup_s3fs()

In [ ]:
import s3fs

In [ ]:
oss = s3fs.S3FileSystem(
      anon=True,
      key="",
      secret="",
      client_kwargs = {"endpoint_url":"https://oss.geodex.org"}
   )

In [ ]:
summoned =oss.ls('gleaner/summoned')
milled =oss.ls('gleaner/milled')

In [ ]:
for repo in summoned:
  fa=oss.ls(repo)
  fnum=len(fa)
  print(f'repo:{repo} has {fnum} files')

repo:gleaner/summoned/amgeo has 5 files
repo:gleaner/summoned/aquadocs has 1 files
repo:gleaner/summoned/bco-dmo has 13221 files
repo:gleaner/summoned/bcodmo has 12484 files
repo:gleaner/summoned/cchdo has 4235 files
repo:gleaner/summoned/designsafe has 352 files
repo:gleaner/summoned/earthchem has 1549 files
repo:gleaner/summoned/edi has 14338 files
repo:gleaner/summoned/geocodes_demo_datasets has 8 files
repo:gleaner/summoned/getiedadataorg has 7206 files
repo:gleaner/summoned/hydroshare has 15297 files
repo:gleaner/summoned/ieda has 7774 files
repo:gleaner/summoned/iedadata has 9347 files
repo:gleaner/summoned/iris has 63 files
repo:gleaner/summoned/linked.earth has 696 files
repo:gleaner/summoned/lipdverse has 698 files
repo:gleaner/summoned/magic has 10725 files
repo:gleaner/summoned/neon has 181 files
repo:gleaner/summoned/ocd has 104951 files
repo:gleaner/summoned/opencoredata has 5894 files
repo:gleaner/summoned/opentopo has 666 files
repo:gleaner/summoned/opentopography has 70

# Finally see how many get into the triplestore that backs 

Each dataset landing page's embeded ld+json jsonld record is put into it's own graph withen the store The graph is labeled with a UniversalResourceName [URN](https://en.wikipedia.org/wiki/Uniform_Resource_Name) which is made with a template of type urn:{type}:{repo}:... So we collect all the URNs, and count how many there are for each repository If we are very lucky, the sitemap count = LD-cache count = this count, but loosing a few from errors does happen

In [ ]:
ec.init_sparql()
df=ec.get_graphs()

In [ ]:
df

,g
0,urn:gleaner2:milled:opentopography:0047cda277b...
1,urn:gleaner2:milled:opentopography:00845ce0c33...
2,urn:gleaner2:milled:opentopography:008970cbb2d...
3,urn:gleaner2:milled:opentopography:00b839b8c22...
4,urn:gleaner2:milled:opentopography:01138abe87a...
...,...
290420,urn:gleaner:prov:r2r:ff91af0a990b49c6fa7cd71f1...
290421,urn:gleaner:prov:r2r:ffc3cd385505936a2df73626c...
290422,urn:gleaner:prov:r2r:ffc9b039d7a4c355a8cadf681...
290423,urn:gleaner:prov:r2r:ffdacfad983e5c8d8b0be4e87...


In [ ]:
#will do in pandas, but 1st pass was w/csv dump, like:
df.to_csv("graphs.csv")

In [ ]:
repoCounts=ec.os_system_("cut -d':' -f3,4 graphs.csv | grep milled | sort | uniq -c |sort -n")
repoCounts

'      3 milled:geocodes_demo_datasets\n      5 milled:amgeo\n     83 milled:iris\n    182 milled:neon\n    697 milled:lipidverse\n    842 milled:usap-dc\n    892 milled:designsafe\n   1379 milled:linked.earth\n   1649 milled:r2r\n   1806 milled:cchdo\n   1823 milled:earthchem\n   2615 milled:opentopography\n   5646 milled:hydroshare\n   7807 milled:edi\n  10793 milled:iedadata\n  12743 milled:magic\n  12902 milled:bco-dmo\n  13379 milled:ucar\n'

In [ ]:
!cut -d':' -f3,4 graphs.csv | grep milled | sort | uniq -c |sort -n 

      3 milled:geocodes_demo_datasets
      5 milled:amgeo
     83 milled:iris
    182 milled:neon
    697 milled:lipidverse
    842 milled:usap-dc
    892 milled:designsafe
   1379 milled:linked.earth
   1649 milled:r2r
   1806 milled:cchdo
   1823 milled:earthchem
   2615 milled:opentopography
   5646 milled:hydroshare
   7807 milled:edi
  10793 milled:iedadata
  12743 milled:magic
  12902 milled:bco-dmo
  13379 milled:ucar


# Put in org info in store, so can query to get a table of the falloff between stages

repo_counts(from sitemap) -> repo_ld_counts(from LD-cache) -> final_counts(in the endpoint)

In [31]:
#get all counts from sitemap/etc caches
repo_counts,repo_ld_counts,final_counts,      repo_df_loc =ec.repos2counts(repos)

Getting http://geocodes.ddns.net/ec/crawl/sitemaps/amgeo.xml
Getting http://geocodes.ddns.net/ec/crawl/sitemaps/cchodo.xml
Getting http://geocodes.ddns.net/ec/crawl/sitemaps/designsafe.xml
Getting http://geocodes.ddns.net/ec/crawl/sitemaps/earthchem.xml
Getting http://geocodes.ddns.net/ec/crawl/sitemaps/geocodes_demo_dataset.xml
Getting http://geocodes.ddns.net/ec/crawl/sitemaps/iris.xml
Getting http://geocodes.ddns.net/ec/crawl/sitemaps/linked.earth.xml
Getting http://geocodes.ddns.net/ec/crawl/sitemaps/lipdverse.xml
Getting http://geocodes.ddns.net/ec/crawl/sitemaps/magic.xml
Getting https://opencoredata.org/sitemap_0.xml
Getting https://opencoredata.org/sitemap_1.xml
Getting https://opencoredata.org/sitemap_3.xml
Getting https://opencoredata.org/sitemap_2.xml
Getting http://geocodes.ddns.net/ec/crawl/sitemaps/r2r.xml
Getting http://geocodes.ddns.net/ec/crawl/sitemaps/ssdb.iodp.xml
Getting http://geocodes.ddns.net/ec/crawl/sitemaps/ucar.xml
Getting http://geocodes.ddns.net/ec/crawl/s

In [32]:
repo_counts

{'amgeo': 5,
 'cchodo': 2520,
 'designsafe': 1638,
 'earthchem': 920,
 'geocodes_demo_dataset': 8,
 'iris': 28,
 'linked.earth': 18634,
 'lipdverse': 704,
 'magic': 4263,
 'opencoredata': 83388,
 'r2r': 44007,
 'ssdb.iodp': 25226,
 'ucar': 17506,
 'unavco': 5643,
 'unidata': 202,
 'usap-dc': 889}

In [33]:
repo_ld_counts

{'amgeo': '5',
 'aquadocs': '1',
 'bco-dmo': '13221',
 'bcodmo': '12484',
 'cchdo': '4235',
 'designsafe': '352',
 'earthchem': '1549',
 'edi': '14338',
 'geocodes_demo_datasets': '8',
 'getiedadataorg': '7206',
 'hydroshare': '15297',
 'ieda': '7774',
 'iedadata': '9347',
 'iris': '63',
 'linked.earth': '696',
 'lipdverse': '698',
 'magic': '10725',
 'neon': '181',
 'ocd': '104951',
 'opencoredata': '5894',
 'opentopo': '666',
 'opentopography': '702',
 'r2r': '1',
 'ssdb': '24124',
 'ssdb.iodp': '25225',
 'ucar': '1',
 'unavco': '13022',
 'usap-dc': '880',
 'wikilinkedearth': '1',
 'wwwbco-dmoorg': '11468',
 'wwwhydroshareorg': '4195'}

In [34]:
final_counts

{'': '7807',
 'adata': '10793',
 'agic': '12743',
 'amgeo': '5',
 'ap-dc': '842',
 'arthchem': '1823',
 'bco-dmo': '12902',
 'car': '13379',
 'cchdo': '1806',
 'geocodes_demo_datasets': '3',
 'hydroshare': '5646',
 'ignsafe': '892',
 'ked.earth': '1379',
 'pentopography': '2615',
 'pidverse': '697',
 'r2r': '1649',
 'ris': '83'}

In [35]:
cmp2=ec.merge_dict_list(repo_counts,repo_ld_counts)
cmp2

defaultdict(list,
            {'amgeo': [5, '5'],
             'aquadocs': ['1'],
             'bco-dmo': ['13221'],
             'bcodmo': ['12484'],
             'cchdo': ['4235'],
             'cchodo': [2520],
             'designsafe': [1638, '352'],
             'earthchem': [920, '1549'],
             'edi': ['14338'],
             'geocodes_demo_dataset': [8],
             'geocodes_demo_datasets': ['8'],
             'getiedadataorg': ['7206'],
             'hydroshare': ['15297'],
             'ieda': ['7774'],
             'iedadata': ['9347'],
             'iris': [28, '63'],
             'linked.earth': [18634, '696'],
             'lipdverse': [704, '698'],
             'magic': [4263, '10725'],
             'neon': ['181'],
             'ocd': ['104951'],
             'opencoredata': [83388, '5894'],
             'opentopo': ['666'],
             'opentopography': ['702'],
             'r2r': [44007, '1'],
             'ssdb': ['24124'],
             'ssdb.iodp': [25226,

In [36]:
cmp_all_stages=ec.merge_dict_list(cmp2,final_counts)
cmp_all_stages

defaultdict(list,
            {'': ['7807'],
             'adata': ['10793'],
             'agic': ['12743'],
             'amgeo': [[5, '5'], '5'],
             'ap-dc': ['842'],
             'aquadocs': [['1']],
             'arthchem': ['1823'],
             'bco-dmo': [['13221'], '12902'],
             'bcodmo': [['12484']],
             'car': ['13379'],
             'cchdo': [['4235'], '1806'],
             'cchodo': [[2520]],
             'designsafe': [[1638, '352']],
             'earthchem': [[920, '1549']],
             'edi': [['14338']],
             'geocodes_demo_dataset': [[8]],
             'geocodes_demo_datasets': [['8'], '3'],
             'getiedadataorg': [['7206']],
             'hydroshare': [['15297'], '5646'],
             'ieda': [['7774']],
             'iedadata': [['9347']],
             'ignsafe': ['892'],
             'iris': [[28, '63']],
             'ked.earth': ['1379'],
             'linked.earth': [[18634, '696']],
             'lipdverse': [[704, 

# Now, Compare some steps against expected values

from sitemap-url to the 2 parts of the LD-cache and final endpoint graph

url -> jsonld (compare w/pre extracted), ntriples (compare w/pre converted),
 
 and w/graph triples
 That can be found hitting the search, via sparql_nb's ec.txt_qry(text), incl timing

 then other related data/tools/etc are also as expected

Can compare each step in isolation, and all the way through

[Some near misses still have same meaning in next step]

In [6]:
ec.first(repo_df_loc)

'amgeo'

read from csv table of urls and expected results, to be kept in github documention  ;either csv or org, both editable

In [ ]:
#read from csv table of urls and expected results, to be kept in github documention  ;either csv or org, both editable

In [8]:
#should/will be:
amgeo=ec.sitemap_all_pages(sitemaps[-1])
amgeo

Getting https://amgeo-dev.colorado.edu/sitemap.xml


0    https://amgeo-dev.colorado.edu/static/data/dat...
1    https://amgeo-dev.colorado.edu/static/data/dat...
2    https://amgeo-dev.colorado.edu/static/data/dat...
3    https://amgeo-dev.colorado.edu/static/data/dat...
4    https://amgeo-dev.colorado.edu/static/data/dat...
Name: loc, dtype: object

In [9]:
amgeo[0]

'https://amgeo-dev.colorado.edu/static/data/data-01-01-2013-12-30-00.html'

In [11]:
ec.init_rdflib()

In [12]:
#do a crawl of a url
ld1=ec.url2jsonLD(amgeo[0])
ld1

{'@context': 'https://schema.org/',
 '@type': 'Dataset',
 'about': 'AMGeO Assimilative Maps',
 'author': {'@type': 'Organization',
  'email': 'amgeo@colorado.edu',
  'logo': 'https://amgeo.colorado.edu/static/img/amgeosmall.svg',
  'name': 'AMGeO'},
 'citation': 'AMGeO Collaboration. (2019). A Collaborative Data Science Platform for the Geospace Community: Assimilative Mapping of Geospace Observations (AMGeO) v1.0.0. Zenodo. https://doi.org/10.5281/zenodo.3564914',
 'description': 'AMGeO helps combine diverse high-latitude geospace observations. The purpose of AMGeO is to make the latest geospace data science tool accessible to scientists and students.',
 'distribution': {'@type': 'DataDownload',
  'contentUrl': 'https://amgeo-dev.colorado.edu/static/data/01-01-2013-12-30-00.nc',
  'encodingFormat': 'application/x-hdf5'},
 'keywords': ['polar ionosphere',
  'magnetosphere-ionosphere coupling',
  'ionospheric electrodynamics'],
 'name': 'AMGeO Assimilative Maps for 2013-01-01T12:30:00',

In [14]:
#retrieve associated gold standard version
ld2=ec.wget2("http://geocodes.ddns.net/ld/amgeo/data-01-01-2013-12-30-00.html.jsonld","ld1gold.jsonld")
ld2

'{\n  "@context": "https://schema.org/",\n  "@type": "Dataset",\n  "about": "AMGeO Assimilative Maps",\n  "author": {\n    "@type": "Organization",\n    "email": "amgeo@colorado.edu",\n    "logo": "https://amgeo.colorado.edu/static/img/amgeosmall.svg",\n    "name": "AMGeO"\n  },\n  "citation": "AMGeO Collaboration. (2019). A Collaborative Data Science Platform for the Geospace Community: Assimilative Mapping of Geospace Observations (AMGeO) v1.0.0. Zenodo. https://doi.org/10.5281/zenodo.3564914",\n  "description": "AMGeO helps combine diverse high-latitude geospace observations. The purpose of AMGeO is to make the latest geospace data science tool accessible to scientists and students.",\n  "distribution": {\n    "@type": "DataDownload",\n    "contentUrl": "https://amgeo-dev.colorado.edu/static/data/01-01-2013-12-30-00.nc",\n    "encodingFormat": "application/x-hdf5"\n  },\n  "keywords": [\n    "polar ionosphere",\n    "magnetosphere-ionosphere coupling",\n    "ionospheric electrodynam

In [24]:
import json
ld1j=json.dumps(ld1, ensure_ascii=True, indent=2)
ec.put_txtfile("ld1.jsonld",ld1j)

2435

In [19]:
#for both jsonld and ntriples comparison, either via direct or ttl file or graph comparison:
ec.diff_nt_g("ld1.jsonld","ld1gold.jsonld") #working on wrapper that prints out: in_both, in_first, in_second 

In [25]:
!wc ld1.jsonld ld1gold.jsonld

  80  211 2435 ld1.jsonld
  81  211 2436 ld1gold.jsonld
 161  422 4871 total


In [30]:
#for this maybe sort keys and forgiving diff
#!diff -wB ld1.jsonld ld1gold.jsonld |wc
#with ntriple comparison have seen ttl help

In [ ]:
#to make sure hasn't changed w/gold stnd, look at df['lastmod']

In [37]:
amgeo.to_csv("amgeo.csv")
!ls -la

total 44
drwxr-xr-x 1 root root 4096 Jul 14 16:47 .
drwxr-xr-x 1 root root 4096 Jul 14 14:15 ..
-rw-r--r-- 1 root root  380 Jul 14 16:47 amgeo.csv
drwxr-xr-x 4 root root 4096 Jul  6 13:21 .config
-rw-r--r-- 1 root root  368 Jul 14 03:50 graph.txt
-rw-r--r-- 1 root root 2436 May  9 18:39 ld1gold.jsonld
-rw-r--r-- 1 root root 2435 Jul 14 14:55 ld1.jsonld
-rw-r--r-- 1 root root 5533 Jul 14 16:16 log
drwxr-xr-x 1 root root 4096 Jul  6 13:22 sample_data
-rw-r--r-- 1 root root  956 Jul 14 03:15 summoned.txt


In [38]:
#can use this as a basis for an expected_urls.csv in https://github.com/MBcode/ec/tree/master/test 
!cat amgeo.csv

,loc
0,https://amgeo-dev.colorado.edu/static/data/data-01-01-2013-12-30-00.html
1,https://amgeo-dev.colorado.edu/static/data/data-01-06-2013-17-30-00.html
2,https://amgeo-dev.colorado.edu/static/data/data-02-06-2013-12-30-00.html
3,https://amgeo-dev.colorado.edu/static/data/data-02-06-2013-13-30-00.html
4,https://amgeo-dev.colorado.edu/static/data/data-03-17-2015-18-00-00.html
